# Consistency Analysis: Lap-to-Lap Variation

This notebook analyzes your driving consistency by measuring variation in braking points, corner speeds, and throttle application across all laps.

## What You'll Find Here

- **Braking Point Consistency**: Box plots showing variation in where you start braking for each corner
- **Corner Speed Consistency**: Box plots showing minimum speed variation through each corner
- **Throttle Acceptance**: The lateral G at which you reach full throttle during corner exit, as a percentage of the corner's peak lateral G
- **Summary Statistics Table**: Mean, standard deviation, min, max, and range for each segment

## How to Interpret the Results

- **Tight box plots** (small range): Consistent performance - you're hitting the same marks each lap
- **Wide box plots** (large range): Inconsistent - opportunity for improvement through practice
- **Outliers** (dots outside whiskers): Unusual laps - could be mistakes, traffic, or experimenting with lines
- **High standard deviation**: Focus area for practice
- **High throttle acceptance %**: Getting on full throttle earlier while still cornering hard - more aggressive exit

## Using Your Own Data

To analyze your own data:

1. **Run the first cell** below to install packages and display the upload widget
2. **Click "Choose File"** to select your `.xrk` or `.xrz` file
3. **Run all remaining cells** to analyze your data

The status indicator will show which file is being used. If you don't upload a file, the sample data will be used.

The notebook will automatically:
- Detect corners and zones from your GPS and pedal data
- Analyze all valid laps (excluding pit laps)
- Generate consistency metrics for each track segment

## Requirements

- GPS data channels (`GPS Latitude`, `GPS Longitude`, `GPS Speed`)
- Brake pressure (`BrakePress`) and throttle (`PPS`)
- Lateral acceleration (`LateralAcc`) for throttle acceptance analysis
- Multiple laps of data for meaningful consistency analysis

**Note:** This notebook works in both JupyterLite (browser) and standard JupyterLab environments.

In [1]:
# Install required packages (needed for JupyterLite, skipped in regular JupyterLab if already installed)
%pip install -q pandas plotly libxrk motorsports-data-notebook jinja2 ipywidgets

# Import core libraries
import pandas as pd
from IPython.display import display

# Visualization libraries
import plotly.express as px
import plotly.graph_objects as go

# Import helper functions
from motorsports_data_notebook.channels import (
    get_best_lap_channels,
    get_top_laps,
)
from motorsports_data_notebook.corners import identify_corners
from motorsports_data_notebook.driver_analysis import find_throttle_acceptance
from motorsports_data_notebook.visualization import (
    format_lap_time,
    visualize_throttle_acceptance,
    show_fig,
)
from motorsports_data_notebook.widgets import FileUpload, load_session
from motorsports_data_notebook.zones import (
    compute_segment_stats,
    create_track_segments,
    detect_zones_averaged,
    get_corner_data,
)

# File picker - upload your own .xrk/.xrz file or use the sample data
file_upload = FileUpload(default_file="CMD_Inferno 86_Fuji GP Sh_Generic testing_a_2248.xrz")
file_upload.display()

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load the data file with derived columns (speed_kmh, distance_m, lap_time)
log = load_session(file_upload.get_file_data())

# Get laps as pandas DataFrame
laps = log.laps.to_pandas()

/home/runner/.cache/pypoetry/virtualenvs/motorsports-data-notebook-lSQWGACw-py3.12/lib/python3.12/site-packages/libxrk/gps.py:343: RuntimeWarning: invalid value encountered in divide
  t = np.maximum(-np.sum(SN * O, axis=1) / np.sum(SN * D, axis=1), 0)
/home/runner/.cache/pypoetry/virtualenvs/motorsports-data-notebook-lSQWGACw-py3.12/lib/python3.12/site-packages/libxrk/gps.py:356: RuntimeWarning: divide by zero encountered in divide
  t = np.maximum(-np.sum(SN * O, axis=1) / np.sum(SN * D, axis=1), 0)
/home/runner/.cache/pypoetry/virtualenvs/motorsports-data-notebook-lSQWGACw-py3.12/lib/python3.12/site-packages/libxrk/gps.py:357: RuntimeWarning: invalid value encountered in multiply
  dist = np.sum(np.square(O + t.reshape((len(t), 1)) * D), axis=1)


division: scan=0.056343, gps=0.024792, group/ch=0.035536 more


In [3]:
# ===== Channel Name Configuration =====
# Different data loggers use different channel names. Configure yours here.
# Print available channels to help identify the correct names:
print("Available channels in this log file:")
print(sorted(log.channels.keys()))

# Required channel mappings - modify values to match YOUR data logger's channel names
# These are defaults for AIM loggers
CHANNEL_NAMES = {
    # GPS channels (required for corner detection)
    "gps_latitude": "GPS Latitude",
    "gps_longitude": "GPS Longitude",
    "gps_speed": "GPS Speed",  # Speed in m/s from GPS
    # Pedal inputs (required for zone detection)
    "throttle": "PPS",  # Throttle position sensor (0-100%)
    "brake": "BrakePress",  # Brake pressure (0-100%)
    # Dynamics (required for throttle acceptance analysis)
    "lateral_g": "LateralAcc",  # Lateral acceleration in G
    "steering": "SteerAngle",  # Steering angle in degrees
}

Available channels in this log file:
['AmbientTemp', 'Baro', 'Best Run Diff', 'Best Today Diff', 'BrakePress', 'BrakeSw', 'CAT1', 'CH', 'ClutchSw', 'ECT', 'External Voltage', 'FL_Ch1', 'FL_Ch2', 'FL_Ch3', 'FL_Ch4', 'FL_Ch5', 'FL_Ch6', 'FL_Ch7', 'FL_Ch8', 'FR_Ch1', 'FR_Ch2', 'FR_Ch3', 'FR_Ch4', 'FR_Ch5', 'FR_Ch6', 'FR_Ch7', 'FR_Ch8', 'GPS Altitude', 'GPS Latitude', 'GPS Longitude', 'GPS Speed', 'Gear', 'InlineAcc', 'IntakeAirT', 'LF_Shock_Pot', 'LR_Shock_Pot', 'Lambda', 'LateralAcc', 'LoggerTemp', 'Luminosity', 'MAP', 'OilTemp', 'PPS', 'PitchRate', 'Predictive Time', 'Prev Lap Diff', 'RF_Shock_Pot', 'RL_Ch1', 'RL_Ch2', 'RL_Ch3', 'RL_Ch4', 'RL_Ch5', 'RL_Ch6', 'RL_Ch7', 'RL_Ch8', 'RPM', 'RR_Ch1', 'RR_Ch2', 'RR_Ch3', 'RR_Ch4', 'RR_Ch5', 'RR_Ch6', 'RR_Ch7', 'RR_Ch8', 'RR_Shock_Pot', 'Ref Lap Diff', 'RollRate', 'SpeedAverage', 'SteerAngle', 'TPMS_ALM_LF', 'TPMS_ALM_LR', 'TPMS_ALM_RF', 'TPMS_ALM_RR', 'TPMS_Press_LF', 'TPMS_Press_LR', 'TPMS_Press_RF', 'TPMS_Press_RR', 'TPMS_Temp_LF', 'TPMS_Tem

In [4]:
# Display lap times table
laps.style.format({"lap_time": format_lap_time})

,num,start_time,end_time,lap_time
0,0,0,150454,2:30.454
1,1,150454,279602,2:09.148
2,2,279602,406240,2:06.638
3,3,406240,532797,2:06.557
4,4,532797,659282,2:06.485
5,5,659282,787773,2:08.491
6,6,787773,913776,2:06.003
7,7,913776,1041397,2:07.621
8,8,1041397,1168322,2:06.925
9,9,1168322,1294676,2:06.354


In [5]:
# Extract best lap channel data using libxrk 0.5.0 methods
best_lap, channels = get_best_lap_channels(
    log, laps, [CHANNEL_NAMES["gps_latitude"], CHANNEL_NAMES["gps_longitude"], "distance_m"]
)

# Filter to best lap and resample to GPS timebase for corner detection
best_lap_num = int(best_lap["num"])
gps_lat_ch = CHANNEL_NAMES["gps_latitude"]
gps_lon_ch = CHANNEL_NAMES["gps_longitude"]
aligned = (
    log.filter_by_lap(best_lap_num)
    .select_channels([gps_lat_ch, gps_lon_ch, "distance_m"])
    .resample_to_channel(gps_lat_ch)
    .channels
)

# Convert to arrays for corner detection
lap_channels = {
    "GPS Latitude": aligned[gps_lat_ch].column(gps_lat_ch).to_numpy(),
    "GPS Longitude": aligned[gps_lon_ch].column(gps_lon_ch).to_numpy(),
    "distance_m": aligned["distance_m"].column("distance_m").to_numpy(),
}

In [6]:
# Identify corners directly from GPS coordinates
corners = identify_corners(
    lat=lap_channels["GPS Latitude"],
    lon=lap_channels["GPS Longitude"],
    threshold=0.003,  # Tuned on Fuji and Sodegaura
    min_corner_length=15,
    min_gap=80,
)

print(f"Found {len(corners)} corners")

Found 10 corners


In [7]:
# Get top laps and detect zones averaged across them
top_laps = get_top_laps(laps, threshold_pct=1.03)

# Detect and average braking/acceleration zones across top laps
braking_zones, accel_zones = detect_zones_averaged(log, top_laps, CHANNEL_NAMES)

print(f"Found {len(braking_zones)} braking zones and {len(accel_zones)} acceleration zones")

Found 7 braking zones and 9 acceleration zones


In [8]:
# Create track segments
track_length = lap_channels["distance_m"][-1]
segments = create_track_segments(corners, braking_zones, accel_zones, track_length)

print(f"Created {len(segments)} track segments")

Created 30 track segments


In [9]:
# Compute per-lap segment statistics
stats_df = compute_segment_stats(log, top_laps, segments, CHANNEL_NAMES)

print(f"Analyzing {len(top_laps)} laps (within 103% of best time)...")
print(f"Computed {len(stats_df)} segment statistics across all laps")

Analyzing 13 laps (within 103% of best time)...
Computed 390 segment statistics across all laps


In [10]:
# Visualize braking consistency
# Show braking point variation for each corner, centered around the mean

braking_stats = stats_df[stats_df["segment_type"] == "braking"].dropna(subset=["braking_point"])

if len(braking_stats) > 0:
    # Calculate deviation from mean braking point for each segment
    braking_stats = braking_stats.copy()
    braking_stats["braking_deviation"] = braking_stats.groupby("segment_name")[
        "braking_point"
    ].transform(lambda x: x - x.mean())

    fig = px.box(
        braking_stats,
        x="segment_name",
        y="braking_deviation",
        title="Braking Point Consistency by Corner (Centered on Mean)",
        labels={"braking_deviation": "Deviation from Mean (m)", "segment_name": "Corner"},
    )
    fig.update_layout(xaxis_tickangle=-45, width=900, height=500)
    # Add a reference line at zero (the mean)
    fig.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5)
    show_fig(fig)
else:
    print("No braking data available")

In [11]:
# Visualize corner minimum speed consistency
corner_stats = stats_df[stats_df["segment_type"] == "corner"].dropna(subset=["min_speed"])

if len(corner_stats) > 0:
    fig = px.box(
        corner_stats,
        x="segment_name",
        y="min_speed",
        title="Minimum Corner Speed Consistency",
        labels={"min_speed": "Min Speed (km/h)", "segment_name": "Corner"},
    )
    fig.update_layout(xaxis_tickangle=-45, width=900, height=500)
    show_fig(fig)
else:
    print("No corner speed data available")

In [12]:
# Compute throttle acceptance for each corner across all laps
# Throttle acceptance = lateral G at sustained full throttle / peak lateral G of corner

# Smoothing window for lateral G calculations and visualization
LATERAL_G_SMOOTHING_WINDOW = 1

# Channels needed for throttle acceptance analysis
throttle_ch = CHANNEL_NAMES["throttle"]
lateral_g_ch = CHANNEL_NAMES["lateral_g"]
THROTTLE_ACCEPTANCE_CHANNELS = ["distance_m", throttle_ch, lateral_g_ch]

throttle_acceptance_stats = []

for idx, lap in top_laps.iterrows():
    lap_num = int(lap["num"])

    # Use libxrk 0.5.0 methods to filter by lap, select channels, and resample
    aligned = (
        log.filter_by_lap(lap_num)
        .select_channels(THROTTLE_ACCEPTANCE_CHANNELS)
        .resample_to_channel("distance_m")
        .channels
    )

    # Build DataFrame with channels + timecodes (timecodes come from aligned tables)
    lap_data = pd.DataFrame(
        {name: aligned[name].column(name).to_numpy() for name in THROTTLE_ACCEPTANCE_CHANNELS}
    )
    # Add timecodes from reference channel's table
    lap_data["timecodes"] = aligned["distance_m"].column("timecodes").to_numpy()

    if len(lap_data) < 10:
        continue

    for corner in corners:
        result = find_throttle_acceptance(
            lap_data, corner, CHANNEL_NAMES, smoothing_window=LATERAL_G_SMOOTHING_WINDOW
        )
        if result is not None:
            throttle_acceptance_stats.append(
                {
                    "corner_name": corner.name,
                    "corner_id": corner.id,
                    "lap_num": lap["num"],
                    "throttle_acceptance_pct": result["throttle_acceptance_pct"],
                    "lateral_g_at_throttle": result["lateral_g_at_throttle"],
                    "peak_lateral_g": result["peak_lateral_g"],
                    "full_throttle_dist": result["full_throttle_dist"],
                }
            )

throttle_acceptance_df = pd.DataFrame(throttle_acceptance_stats)
print(f"Computed throttle acceptance for {len(throttle_acceptance_df)} corner/lap combinations")

Computed throttle acceptance for 127 corner/lap combinations


In [13]:
# Visualize throttle acceptance consistency
# Shows at what percentage of peak lateral G the driver reaches full throttle

if len(throttle_acceptance_df) > 0:
    fig = px.box(
        throttle_acceptance_df,
        x="corner_name",
        y="throttle_acceptance_pct",
        title="Throttle Acceptance by Corner (% of Peak Lateral G at Full Throttle)",
        labels={
            "throttle_acceptance_pct": "Throttle Acceptance (%)",
            "corner_name": "Corner",
        },
    )
    fig.update_layout(xaxis_tickangle=-45, width=900, height=500)
    # Add reference lines
    fig.add_hline(
        y=100,
        line_dash="dash",
        line_color="red",
        opacity=0.5,
        annotation_text="100% = Full throttle at peak G",
    )
    show_fig(fig)
else:
    print("No throttle acceptance data available")

In [14]:
# Throttle acceptance summary statistics table
if len(throttle_acceptance_df) > 0:
    throttle_summary = []
    for corner_name in throttle_acceptance_df["corner_name"].unique():
        corner_data = throttle_acceptance_df[throttle_acceptance_df["corner_name"] == corner_name]
        throttle_summary.append(
            {
                "Corner": corner_name,
                "Mean (%)": corner_data["throttle_acceptance_pct"].mean(),
                "Std (%)": corner_data["throttle_acceptance_pct"].std(),
                "Min (%)": corner_data["throttle_acceptance_pct"].min(),
                "Max (%)": corner_data["throttle_acceptance_pct"].max(),
                "Range (%)": corner_data["throttle_acceptance_pct"].max()
                - corner_data["throttle_acceptance_pct"].min(),
                "N": len(corner_data),
            }
        )

    throttle_summary_df = pd.DataFrame(throttle_summary)
    display(
        throttle_summary_df.style.format(
            {
                "Mean (%)": "{:.1f}",
                "Std (%)": "{:.1f}",
                "Min (%)": "{:.1f}",
                "Max (%)": "{:.1f}",
                "Range (%)": "{:.1f}",
            }
        )
    )
else:
    print("No throttle acceptance data available")

,Corner,Mean (%),Std (%),Min (%),Max (%),Range (%),N
0,Turn 1,77.7,17.9,34.8,97.9,63.2,13
1,Turn 2,72.6,11.5,51.6,95.9,44.3,13
2,Turn 3,79.8,6.2,69.5,89.5,19.9,13
3,Turn 4,77.3,9.2,61.9,91.6,29.6,13
4,Turn 5,86.1,5.7,76.1,95.3,19.1,13
5,Turn 7,79.9,11.2,53.2,92.8,39.5,13
6,Turn 8,69.3,6.4,57.8,81.9,24.1,13
7,Turn 9,51.0,26.8,3.8,86.5,82.7,12
8,Turn 10,65.9,9.5,44.7,76.8,32.1,13
9,Turn 6,83.6,9.6,64.5,92.5,28.0,11


In [15]:
# Visualize throttle acceptance concept for Turn 1 (best lap)
# Shows throttle, brake, steering, and lateral G over distance with reference lines

turn1 = corners[0]

# Use libxrk 0.5.0 methods to get channels for best lap
best_lap_num = int(best_lap["num"])
best_lap_aligned = (
    log.filter_by_lap(best_lap_num)
    .select_channels(THROTTLE_ACCEPTANCE_CHANNELS)
    .resample_to_channel("distance_m")
    .channels
)
best_lap_df = pd.DataFrame(
    {name: best_lap_aligned[name].column(name).to_numpy() for name in THROTTLE_ACCEPTANCE_CHANNELS}
)
# Add timecodes from reference channel's table
best_lap_df["timecodes"] = best_lap_aligned["distance_m"].column("timecodes").to_numpy()

turn1_result = find_throttle_acceptance(
    best_lap_df, turn1, CHANNEL_NAMES, smoothing_window=LATERAL_G_SMOOTHING_WINDOW
)
assert turn1_result is not None, f"Could not compute throttle acceptance for {turn1.name}"

# Get corner data for best lap (receives pre-filtered log)
brake_ch = CHANNEL_NAMES["brake"]
steering_ch = CHANNEL_NAMES["steering"]
CORNER_VIZ_CHANNELS = ["distance_m", throttle_ch, brake_ch, lateral_g_ch, steering_ch]
lap_log = log.filter_by_lap(best_lap_num)
corner_data = get_corner_data(lap_log, turn1, CORNER_VIZ_CHANNELS, margin=50)

# Compute smoothed lateral G
lateral_g_smooth = (
    corner_data[lateral_g_ch]
    .abs()
    .rolling(window=LATERAL_G_SMOOTHING_WINDOW, center=True, min_periods=1)
    .mean()
)

fig = visualize_throttle_acceptance(
    distance=corner_data["distance_m"],
    throttle=corner_data[throttle_ch],
    lateral_g=lateral_g_smooth,
    corner=turn1,
    throttle_acceptance_result=turn1_result,
    brake=corner_data.get(brake_ch),
    steering=corner_data.get(steering_ch),
)
show_fig(fig)

In [16]:
# Summary statistics table
def compute_summary_stats(stats_df):
    """Compute summary statistics for each segment across all laps."""
    summary = []

    # Braking segments
    for seg_name in stats_df[stats_df["segment_type"] == "braking"]["segment_name"].unique():
        seg_data = stats_df[
            (stats_df["segment_name"] == seg_name) & stats_df["braking_point"].notna()
        ]
        if len(seg_data) > 0:
            summary.append(
                {
                    "Segment": seg_name,
                    "Type": "Braking",
                    "Metric": "Braking Point (m)",
                    "Mean": seg_data["braking_point"].mean(),
                    "Std": seg_data["braking_point"].std(),
                    "Min": seg_data["braking_point"].min(),
                    "Max": seg_data["braking_point"].max(),
                    "Range": seg_data["braking_point"].max() - seg_data["braking_point"].min(),
                    "N": len(seg_data),
                }
            )

    # Corner segments
    for seg_name in stats_df[stats_df["segment_type"] == "corner"]["segment_name"].unique():
        seg_data = stats_df[(stats_df["segment_name"] == seg_name) & stats_df["min_speed"].notna()]
        if len(seg_data) > 0:
            summary.append(
                {
                    "Segment": seg_name,
                    "Type": "Corner",
                    "Metric": "Min Speed (km/h)",
                    "Mean": seg_data["min_speed"].mean(),
                    "Std": seg_data["min_speed"].std(),
                    "Min": seg_data["min_speed"].min(),
                    "Max": seg_data["min_speed"].max(),
                    "Range": seg_data["min_speed"].max() - seg_data["min_speed"].min(),
                    "N": len(seg_data),
                }
            )

    return pd.DataFrame(summary)


summary_df = compute_summary_stats(stats_df)
summary_df.style.format(
    {"Mean": "{:.1f}", "Std": "{:.1f}", "Min": "{:.1f}", "Max": "{:.1f}", "Range": "{:.1f}"}
)

,Segment,Type,Metric,Mean,Std,Min,Max,Range,N
0,Turn 1 Braking,Braking,Braking Point (m),576.7,5.0,572.4,585.7,13.3,13
1,Turn 2 Braking,Braking,Braking Point (m),1214.8,4.6,1211.0,1224.8,13.7,13
2,Turn 4 Braking,Braking,Braking Point (m),1903.9,3.3,1901.4,1912.8,11.4,13
3,Turn 6 Braking,Braking,Braking Point (m),2686.3,2.3,2684.1,2692.1,8.0,13
4,Turn 7 Braking,Braking,Braking Point (m),2686.3,2.3,2684.1,2692.1,8.0,13
5,Turn 1,Corner,Min Speed (km/h),64.3,2.7,58.3,68.4,10.1,13
6,Turn 2,Corner,Min Speed (km/h),127.7,4.6,115.8,132.7,16.9,13
7,Turn 3,Corner,Min Speed (km/h),136.7,2.8,131.1,142.1,11.0,13
8,Turn 4,Corner,Min Speed (km/h),87.3,2.9,82.2,92.0,9.8,13
9,Turn 5,Corner,Min Speed (km/h),137.6,1.8,134.7,140.1,5.4,13
